In [63]:
from transformers import (
    BertTokenizerFast,
    BertForMaskedLM,
    AutoModelForSequenceClassification,
)

In [64]:
import pandas as pd

pd.set_option("display.max_columns", None)

need_cols = [
    "PRS",
    "sex",
    "height",
    "weight",
    "BSA",
    # "genotype_array",
    "age",
    # "PC1",
    # "PC2",
    # "PC3",
    # "PC4",
    # "PC5",
    # "PC6",
    # "PC7",
    # "PC8",
    # "PC9",
    # "PC10",
    # "assessment_center",
    # "age_squared",
    "BMI",
    "dbp_a",
    "sbp_a",
    "tc_a",
    "tg_a",
    "hdl_a",
    "ldl_a",
    "dbp",
    "sbp",
    "tc",
    "tg",
    "hdl",
    "ldl",
    "apob",
    "cad",
    "mi",
    "crp",
    "eGFR",
    "Cr",
    "CAD",
    "cad_age",
    "death_age",
    # "recuit_age",
    # "year_of_cad_after_recuit",
    # "survival_time",
    # "is_earily_CAD",
    "incident_cad",
    # "cad_status",
    # "prevalent_cad",
    "smoking",
    "prevalent_diabetes",
    "Arrhythmia",
    "Stroke",
    "Peripheral_vascular_disease",
    "Hypertension",
    "future_cad_time_years",
]
train_imputed = pd.read_pickle("result/part1/train_imputed.pkl").set_index("eid")[
    need_cols
]
test_imputed = pd.read_pickle("result/part1/test_imputed.pkl").set_index("eid")[
    need_cols
]


train_data = pd.read_pickle(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/train.pkl"
)

test_data = pd.read_pickle(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/eval.pkl"
)
protein_cols = test_data.columns.tolist()

In [65]:
train_data.index = train_data.index.astype(int)
test_data.index = test_data.index.astype(int)
train_imputed.index = train_imputed.index.astype(int)
test_imputed.index = test_imputed.index.astype(int)

total = pd.concat([train_imputed, test_imputed])

train_data = train_data.merge(total, left_index=True, right_index=True, how="inner")
test_data = test_data.merge(total, left_index=True, right_index=True, how="inner")
test_data

C3      KLK7     GCHFR    NHLRC3     APOD     GAPDH   TP53I3  \
eid                                                                           
1982486  0.040385  0.007345  0.057373 -0.000238  0.02491 -0.065304  0.02669   
5937359  0.040385  0.007345  0.057373 -0.000238  0.02491 -0.065304  0.02669   
3737374 -0.557200 -0.499600  0.575800 -0.589900 -0.49640  0.118850  0.17070   
5525566  0.141400 -0.525100 -0.338000 -0.233100 -0.02070  0.077700  0.33230   
1828132  0.386100 -0.094400  0.546200 -0.163200  0.02491  0.088000  0.02180   
...           ...       ...       ...       ...      ...       ...      ...   
2200051 -0.190400 -0.538750  0.113200  0.164100 -0.58220 -0.072700 -0.72350   
4095573 -0.086900 -0.232300 -0.814900 -0.269100  0.00000 -0.551450 -0.70850   
5600706  0.285500 -0.227300  0.352500  0.293800 -0.27210  0.109750  0.63860   
5763542  0.009200  0.542700  0.574100  0.380700  0.23640  0.256100  0.17830   
4813271  0.217900 -0.594800 -0.241700  0.561000 -0.94230 -0.104550 -0.43810   

             CPA4     ANXA2     GRSF1      IL25      HMMR    MRPL52    PAIP2B  \
eid                                                                             
1982486 -0.009212  0.073499  0.082175 -0.019463  0.054938 -0.003103  0.029678   
5937359 -0.009212  0.073499  0.082175 -0.019463  0.054938 -0.003103  0.029678   
3737374 -0.074400  0.032050 -0.209000 -0.293700 -0.022350 -0.185700  0.074850   
5525566  0.144400  2.277800  1.125300  0.274500  0.119100  0.231100  1.728450   
1828132  0.194000 -0.261200 -0.168600 -0.368600 -0.163300 -0.210600  0.592150   
...           ...       ...       ...       ...       ...       ...       ...   
2200051 -0.216350 -0.083400 -0.599400 -0.401000  0.165300  0.017250 -0.156000   
4095573 -0.110800  0.381650 -0.291400 -0.692600 -0.299450  0.107300 -0.382750   
5600706 -0.117400  0.588050  0.786700 -0.319200  0.246350  0.054600 -0.907550   
5763542  1.040900  0.240400 -0.623200  0.057400  0.143000 -0.282700  0.215550   
4813271  0.024800  0.655950  0.082175 -0.019463  0.054938 -0.003103  0.029678   

           THAP12       FOS      FGF9    PITHD1     THSD1    PTGES2  \
eid                                                                   
1982486  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   
5937359  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   
3737374 -0.762200  0.200200  0.049900  0.160550  0.065800 -0.593400   
5525566  0.315300  0.588700  0.552700  0.251100  0.595200  0.680000   
1828132 -0.051400 -0.323600  0.208400  0.254800 -0.225800 -0.163900   
...           ...       ...       ...       ...       ...       ...   
2200051 -0.172500  0.120900  0.359000 -0.112200  0.364750 -0.327700   
4095573  0.310200 -0.408000 -0.419600 -0.384150 -0.211700 -0.637500   
5600706  0.181400 -0.652800 -0.583900 -0.423050 -0.251400  0.516100   
5763542  0.029300  0.152000  0.337800  2.840500  0.492900 -0.149500   
4813271  0.174497 -0.013819 -0.021022  0.161046  0.104000 -0.502000   

         DEFB103A_DEFB103B    ATP1B4     CYB5A     UNC79   SLC34A3    TAGLN3  \
eid                                                                            
1982486           0.046926  0.043311  0.082135  0.058686  0.032686 -0.012728   
5937359           0.046926  0.043311  0.082135  0.058686  0.032686 -0.012728   
3737374          -0.443000 -0.267950  0.104000 -0.044500  0.482050 -0.220800   
5525566           0.041700  0.169150 -0.276800  0.464800  0.310450 -0.171100   
1828132           0.380800 -0.374650  0.800100 -0.025900  0.472350  0.673200   
...                    ...       ...       ...       ...       ...       ...   
2200051          -0.176600  0.226300  0.000000 -0.665600  0.070250 -0.037900   
4095573           0.142900  0.723150  1.473600  1.145700  0.348250  0.182400   
5600706          -0.165900 -0.076450 -0.244500  0.048000 -0.021050 -0.434400   
5763542           0.046900 -0.116050  0.038800  0.252300  0.149350  0.321800   
4813271          -0.065400  0.043311  0.082135  0

In [66]:
from datasets import Dataset
from collections import defaultdict
from multiprocessing import cpu_count
import numpy as np
from tqdm.autonotebook import tqdm


# def castForDF2Dataset(df):
#     dtypes = df.dtypes.to_dict()
#     supported_dtypes = [
#         np.int32,
#         np.float32,
#         np.float64,
#         np.int64,
#         np.float16,
#         np.int16,
#         np.int8,
#         np.uint8,
#         np.uint16,
#         np.uint32,
#         np.uint64,
#         np.bool_,
#     ]
#     for col in df.columns:
#         dtype = df[col].dtypes
#         if dtype in supported_dtypes:
#             continue

#         print(f"{col} has unsupported dtype {dtypes[col]} cast to string")
#         df[col] = df[col].astype(str)

#     return df


def df2dataset(
    df,
    tokenizer,
    max_length=2048,
    features=None,
    attr=None,
    tgt=None,
    nproc=4,
    batch_size=512,
):
    res = defaultdict(list)

    if features is None and attr is None:
        features = df.columns.tolist()

    ## drop features not in tokenizer

    tokens = list(tokenizer.vocab.keys())
    not_in_list = []
    for col in features:
        if col.lower() not in tokens:  # lower case
            not_in_list.append(col)
    print(
        f"Total {len(not_in_list)} proteins not in tokens will drop them , part of them are : {not_in_list[:10]}"
    )
    print(not_in_list)
    df.drop(columns=not_in_list, inplace=True)
    ### update features
    features = [col for col in features if col in df.columns]

    ## to dict
    for idx, row in tqdm(
        df.iterrows(), total=df.shape[0], desc="processing df to dict"
    ):

        features_row = row[features]
        attr_row = row[attr]

        ranked_row = features_row.sort_values(ascending=False).dropna()

        res["eid"].append(ranked_row.name)
        res["proteins"].append(" ".join(ranked_row.index.tolist()))
        res["values"].append(ranked_row.values.tolist())
        res["length"].append(len(ranked_row))
        res["attr"].append(attr_row.index.tolist())
        res["attr_values"].append(attr_row.values.tolist())
        if tgt is not None:
            res["label"].append(row[tgt])
            res["label_name"].append(tgt)

    ## to dataset
    dataset = Dataset.from_dict(res)

    ## tokenize data
    def group_texts(examples):

        tokenized_inputs = tokenizer(
            examples["proteins"],
            return_special_tokens_mask=True,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            truncation_strategy="only_first",
        )

        return tokenized_inputs

    dataset = dataset.map(
        group_texts,
        batched=True,
        remove_columns=["proteins"],
        batch_size=batch_size,
        num_proc=min(cpu_count(), 12) if nproc is None else nproc,
    )
    return dataset

In [67]:
test_data.head(200)

C3      KLK7     GCHFR    NHLRC3     APOD     GAPDH   TP53I3  \
eid                                                                           
1982486  0.040385  0.007345  0.057373 -0.000238  0.02491 -0.065304  0.02669   
5937359  0.040385  0.007345  0.057373 -0.000238  0.02491 -0.065304  0.02669   
3737374 -0.557200 -0.499600  0.575800 -0.589900 -0.49640  0.118850  0.17070   
5525566  0.141400 -0.525100 -0.338000 -0.233100 -0.02070  0.077700  0.33230   
1828132  0.386100 -0.094400  0.546200 -0.163200  0.02491  0.088000  0.02180   
...           ...       ...       ...       ...      ...       ...      ...   
1655152  0.313600 -0.407700  0.045200 -0.111300 -0.22340 -0.180000 -0.90370   
3195605 -0.150200 -0.270700  0.200600  0.098700  0.53620 -1.136600 -0.78320   
4880045 -0.367900  0.007345  0.057373 -0.000238  0.02491 -0.197350 -0.06000   
4885827  0.053000 -0.762450  0.062400  0.404600  0.03070 -0.033200  0.21120   
1168642 -0.240700  0.169400 -0.411200 -0.175200 -0.20450  0.123950  0.61630   

             CPA4     ANXA2     GRSF1      IL25      HMMR    MRPL52    PAIP2B  \
eid                                                                             
1982486 -0.009212  0.073499  0.082175 -0.019463  0.054938 -0.003103  0.029678   
5937359 -0.009212  0.073499  0.082175 -0.019463  0.054938 -0.003103  0.029678   
3737374 -0.074400  0.032050 -0.209000 -0.293700 -0.022350 -0.185700  0.074850   
5525566  0.144400  2.277800  1.125300  0.274500  0.119100  0.231100  1.728450   
1828132  0.194000 -0.261200 -0.168600 -0.368600 -0.163300 -0.210600  0.592150   
...           ...       ...       ...       ...       ...       ...       ...   
1655152 -0.423200 -0.601600 -0.761300 -0.197000  0.113200 -0.026100  0.229450   
3195605  0.062400  0.093200 -1.501900  0.273100  0.967600  0.002200  0.160650   
4880045 -0.140700  0.165750 -0.891500  0.043200  0.450250 -0.003200 -0.373250   
4885827 -0.906050  0.301200  0.351100 -0.185500 -0.387000 -0.262250 -0.268400   
1168642  0.353700 -0.016150  1.352500 -0.190000 -0.262150 -0.235200  0.118750   

           THAP12       FOS      FGF9    PITHD1     THSD1    PTGES2  \
eid                                                                   
1982486  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   
5937359  0.174497 -0.013819 -0.021022  0.161046  0.007754  0.036311   
3737374 -0.762200  0.200200  0.049900  0.160550  0.065800 -0.593400   
5525566  0.315300  0.588700  0.552700  0.251100  0.595200  0.680000   
1828132 -0.051400 -0.323600  0.208400  0.254800 -0.225800 -0.163900   
...           ...       ...       ...       ...       ...       ...   
1655152 -0.014300  0.088000  0.197800  0.552700  0.007754  0.036311   
3195605  0.190100  0.269600  0.327600  2.031200 -0.341300 -0.990300   
4880045  0.137800 -0.491600 -0.148800  2.306850  0.168800 -0.900300   
4885827  0.136900  0.191000  0.045300 -0.305100  0.255850  0.046600   
1168642  0.101800  0.604400  0.074500  0.126650  0.037100  0.901600   

         DEFB103A_DEFB103B    ATP1B4     CYB5A     UNC79   SLC34A3    TAGLN3  \
eid                                                                            
1982486           0.046926  0.043311  0.082135  0.058686  0.032686 -0.012728   
5937359           0.046926  0.043311  0.082135  0.058686  0.032686 -0.012728   
3737374          -0.443000 -0.267950  0.104000 -0.044500  0.482050 -0.220800   
5525566           0.041700  0.169150 -0.276800  0.464800  0.310450 -0.171100   
1828132           0.380800 -0.374650  0.800100 -0.025900  0.472350  0.673200   
...                    ...       ...       ...       ...       ...       ...   
1655152          -0.116300  0.159550  0.766900 -0.260100 -0.011350 -0.717100   
3195605           0.282700  0.571050 -0.415400 -0.571100  0.135050 -0.062900   
4880045          -0.102100 -0.187250  0.000100 -0.199900  0.429150  0.522800   
4885827          -0.091400 -0.564500  0.019400  0.400100 -0.366950 -0.342500   
1168642           0.096400  0.092850  0.097900  0

In [68]:
# # because some errors of tokenizer is lower case, so we need to convert all to lower case
# train_data.columns = train_data.columns.str.lower()
# test_data.columns = test_data.columns.str.lower()

# protein_cols = [col.lower() for col in protein_cols]
# need_cols = [col.lower() for col in need_cols]

In [70]:
tokenizer = BertTokenizerFast.from_pretrained(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/tokenizer"
)

In [71]:
test_dataset = df2dataset(
    test_data.head(200),
    tokenizer,
    features=protein_cols,
    attr=need_cols,
    tgt="incident_cad",
    # max_length=128,
)
test_dataset

Total 0 proteins not in tokens will drop them , part of them are : []
[]


/tmp/ipykernel_3433/1057266987.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=not_in_list, inplace=True)


processing df to dict:   0%|          | 0/200 [00:00<?, ?it/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['eid', 'values', 'length', 'attr', 'attr_values', 'label', 'label_name', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 200
})

In [72]:
test_dataset[0]["input_ids"]

[2262,
 389,
 1326,
 88,
 1703,
 238,
 73,
 2441,
 2670,
 332,
 60,
 2644,
 1037,
 740,
 990,
 1343,
 2126,
 512,
 487,
 1219,
 1415,
 793,
 649,
 1776,
 1212,
 690,
 346,
 2546,
 2135,
 1096,
 959,
 2860,
 1381,
 2800,
 215,
 618,
 2702,
 1949,
 2645,
 530,
 18,
 651,
 1557,
 1828,
 794,
 2340,
 2256,
 739,
 1836,
 1061,
 361,
 134,
 844,
 2635,
 2236,
 462,
 1498,
 2573,
 1706,
 36,
 1679,
 2503,
 2039,
 2100,
 1726,
 1746,
 264,
 2266,
 2343,
 2589,
 2275,
 2567,
 852,
 2575,
 2883,
 2877,
 2875,
 2876,
 2866,
 2804,
 1098,
 2679,
 1243,
 1135,
 1152,
 1456,
 911,
 2465,
 2842,
 1576,
 1142,
 2109,
 1553,
 1930,
 2257,
 2028,
 764,
 809,
 750,
 1263,
 585,
 1377,
 13,
 2397,
 533,
 2113,
 439,
 1560,
 1747,
 2790,
 2448,
 502,
 2768,
 1539,
 1874,
 1210,
 1590,
 2771,
 2450,
 1881,
 539,
 874,
 1300,
 2424,
 2344,
 2719,
 1418,
 1260,
 327,
 293,
 2511,
 2582,
 1310,
 2830,
 2896,
 1197,
 1658,
 994,
 1432,
 451,
 1667,
 584,
 2500,
 882,
 2605,
 2278,
 497,
 1524,
 2255,
 561,
 924

In [9]:
from transformers import DataCollatorForLanguageModeling


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [36]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000"
)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/checkpoint-60000 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(2916, 256, padding_idx=0)
      (position_embeddings): Embedding(2916, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, e

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="test",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

# trainer.train()

/home/xutingfeng/miniforge3/envs/rapids-24.02/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
for batch in trainer.get_train_dataloader():
    break

In [13]:
batch["labels"]

tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, 1586, -100]], device='cuda:0')

In [14]:
model.eval()
o = model(**batch)
o

MaskedLMOutput(loss=tensor(7.2867, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[-1.2068e+01, -6.5331e+00, -6.3259e+00,  ..., -2.0682e-01,
           5.5067e-01,  3.2887e+00],
         [-1.2174e+01, -7.0543e+00, -7.0550e+00,  ..., -5.3003e-02,
          -2.6013e-01,  3.2283e+00],
         [-9.9081e+00, -6.4985e+00, -6.0032e+00,  ...,  1.3337e+00,
           6.0500e-01,  3.6630e+00],
         ...,
         [-1.2361e+01, -7.4547e+00, -7.2956e+00,  ..., -1.1503e+00,
          -1.5066e-03,  1.6074e-01],
         [-1.1565e+01, -7.8297e+00, -7.7378e+00,  ..., -9.6790e-01,
          -8.1222e-01,  9.4881e-01],
         [-1.1099e+01, -7.8807e+00, -7.6688e+00,  ..., -7.0626e-01,
          -5.2311e-01, -2.8745e-01]],

        [[-1.3288e+01, -6.0509e+00, -5.8190e+00,  ...,  2.5689e+00,
           1.1551e-01,  2.3039e+00],
         [-1.0364e+01, -5.0867e+00, -5.0685e+00,  ...,  2.7335e+00,
           4.2778e-02,  2.4439e+00],
         [-1.0629e+01, -5.7077e+00, -5.5526e+00,  ...,  

In [22]:
batch["labels"][:, :20]

tensor([[-100, -100, -100, -100, -100, -100, -100, 1340, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100, -100, 1467, -100, -100, -100, -100, -100, 1037, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]], device='cuda:0')

In [21]:
import torch

torch.argmax(o["logits"], dim=2)[:, :20]

tensor([[1153, 1121, 1882,  106, 1642,  817, 1907, 1509, 1885, 1888, 1729, 1350,
         1617, 2625,  260,  950,  179, 2317,  798, 2650],
        [1380, 1470, 2537, 2537, 1888, 1025, 1564,  160, 1433, 1345, 1326,  364,
         2173,  640,  560, 1516, 1798, 2220, 1949,   67]], device='cuda:0')

In [20]:
batch["input_ids"][:, :20]

tensor([[ 901, 1121, 1882,  106, 1642,  817, 1907,    3, 1885, 1888, 1729, 1350,
         1617, 2625,  260,  950,  179, 2317,  798, 2650],
        [ 901, 1470, 2537,    3, 1888, 1025, 1564,  160, 1433, 1345, 1326,  364,
         2173,  640,  560, 1516, 1798, 2220, 1949,   67]], device='cuda:0')

In [23]:
tokenizer.decode(batch["input_ids"][0, :20])

'egfr gast nphs1 alpp lypla2 dlgap5 ntprobnp [MASK] nppb nptn mmp12 iglon5 lrrfip1 tcl1a bgn epo arhgef5 ruvbl1 defb118 tg'

In [24]:
tokenizer.decode(torch.argmax(o["logits"], dim=2)[0, :20])

'gh2 gast nphs1 alpp lypla2 dlgap5 ntprobnp klk1 nppb nptn mmp12 iglon5 lrrfip1 tcl1a bgn epo arhgef5 ruvbl1 defb118 tg'

In [51]:
o

MaskedLMOutput(loss=tensor(7.2867, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[-1.2068e+01, -6.5331e+00, -6.3259e+00,  ..., -2.0682e-01,
           5.5067e-01,  3.2887e+00],
         [-1.2174e+01, -7.0543e+00, -7.0550e+00,  ..., -5.3003e-02,
          -2.6013e-01,  3.2283e+00],
         [-9.9081e+00, -6.4985e+00, -6.0032e+00,  ...,  1.3337e+00,
           6.0500e-01,  3.6630e+00],
         ...,
         [-1.2361e+01, -7.4547e+00, -7.2956e+00,  ..., -1.1503e+00,
          -1.5066e-03,  1.6074e-01],
         [-1.1565e+01, -7.8297e+00, -7.7378e+00,  ..., -9.6790e-01,
          -8.1222e-01,  9.4881e-01],
         [-1.1099e+01, -7.8807e+00, -7.6688e+00,  ..., -7.0626e-01,
          -5.2311e-01, -2.8745e-01]],

        [[-1.3288e+01, -6.0509e+00, -5.8190e+00,  ...,  2.5689e+00,
           1.1551e-01,  2.3039e+00],
         [-1.0364e+01, -5.0867e+00, -5.0685e+00,  ...,  2.7335e+00,
           4.2778e-02,  2.4439e+00],
         [-1.0629e+01, -5.7077e+00, -5.5526e+00,  ...,  

In [17]:
from datasets import load_from_disk

test_data = load_from_disk(
    "/home/xutingfeng/ukb/project/ppp_prediction/result/dl/ProteomicsBERT/test"
)
test_data

Dataset({
    features: ['eid', 'values', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 4000
})

In [18]:
test_data = test_data.select_columns(
    ["input_ids", "token_type_ids", "attention_mask", "special_tokens_mask"]
)
test_data

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 4000
})

In [19]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(2916, 256, padding_idx=0)
      (position_embeddings): Embedding(2916, 256)
      (token_type_embeddings): Embedding(2, 256)
      (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=256, out_features=256, bias=True)
              (LayerNorm): LayerNorm((256,), eps=1e-12, elementwise_aff

In [50]:
torch.argmax(o["logits"][0, :3], dim=1)

tensor([1153, 1121, 1882], device='cuda:0')

In [46]:
o["logits"][0, :3].shape

torch.Size([3, 2916])